In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pickle
import time
from tqdm.notebook import tqdm

import torch
torch.set_default_tensor_type(torch.DoubleTensor)

from spatial_scene_grammars.constraints import *
from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.visualization import *
from spatial_scene_grammars_examples.dish_bin.grammar import *
from spatial_scene_grammars.parsing import *
from spatial_scene_grammars.sampling import *
from spatial_scene_grammars.parameter_estimation import *
from spatial_scene_grammars.dataset import *

import meshcat
import meshcat.geometry as meshcat_geom

Generating from folder  sink/plates_cups_and_bowls/plates
sink:plates_cups_and_bowls:plates:Threshold_Bistro_Ceramic_Dinner_Plate_Ruby_Ring :  <class 'spatial_scene_grammars_examples.dish_bin.grammar.sink:plates_cups_and_bowls:plates:Threshold_Bistro_Ceramic_Dinner_Plate_Ruby_Ring'>
sink:plates_cups_and_bowls:plates:Ecoforms_Plate_S20Avocado :  <class 'spatial_scene_grammars_examples.dish_bin.grammar.sink:plates_cups_and_bowls:plates:Ecoforms_Plate_S20Avocado'>
sink:plates_cups_and_bowls:plates:Room_Essentials_Salad_Plate_Turquoise :  <class 'spatial_scene_grammars_examples.dish_bin.grammar.sink:plates_cups_and_bowls:plates:Room_Essentials_Salad_Plate_Turquoise'>
sink:plates_cups_and_bowls:plates:Ecoforms_Plant_Plate_S11Turquoise :  <class 'spatial_scene_grammars_examples.dish_bin.grammar.sink:plates_cups_and_bowls:plates:Ecoforms_Plant_Plate_S11Turquoise'>
Generating from folder  sink/plates_cups_and_bowls/cups
sink:plates_cups_and_bowls:cups:Cole_Hardware_Mug_Classic_Blue :  <class '

In [2]:
if 'vis' not in globals():
    vis = meshcat.Visualizer()
vis.delete()
base_url = "http://127.0.0.1"
meshcat_url = base_url + ":" + vis.url().split(":")[-1]
print("Meshcat url: ", meshcat_url)
'''
from IPython.display import HTML
HTML("""
    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">
    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>
</div>
""".format(url=meshcat_url))
'''

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Meshcat url:  http://127.0.0.1:7001/static/


'\nfrom IPython.display import HTML\nHTML("""\n    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">\n    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>\n</div>\n""".format(url=meshcat_url))\n'

In [7]:
# Draw a random sample from the grammar with its initial params and visualize it.
#torch.random.manual_seed(5)
grammar = SpatialSceneGrammar(
    root_node_type = DishBin,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.5, 0., 0.]))
)
grammar.load_state_dict(torch.load("fit_grammar.torch"))

class InBinConstraint(Constraint):
    # XY coord of each object inside .56 x .83 dish bin
    def __init__(self):
        lb = torch.tensor([-0.56/2+0.1, -0.83/2+0.1, 0.])
        ub = torch.tensor([0.56/2-0.1, 0.83/2-0.1, 1.])
        super().__init__(
            lower_bound=lb,
            upper_bound=ub
        )
    def eval(self, scene_tree):
        xys = []
        bin_pos = scene_tree.find_nodes_by_type(DishBin)[0].translation
        for node in scene_tree.find_nodes_by_type(ObjectModel):
            xys.append(node.translation - bin_pos)
        return torch.stack(xys, axis=0)
    def add_to_ik_prog(self, scene_tree, ik, mbp, mbp_context, node_to_free_body_ids_map):
        bin_pos = scene_tree.find_nodes_by_type(DishBin)[0].translation.detach().numpy()
        for node in scene_tree.find_nodes_by_type(ObjectModel):
            for body_id in node_to_free_body_ids_map[node]:
                body = mbp.get_body(body_id)
                print(bin_pos + self.lower_bound.detach().numpy(), bin_pos + self.upper_bound.detach().numpy())
                ik.AddPositionConstraint(
                    body.body_frame(), np.zeros(3),
                    mbp.world_frame(),
                    bin_pos + self.lower_bound.detach().numpy(),
                    bin_pos + self.upper_bound.detach().numpy()
                )
constraints = [
    InBinConstraint()
]
torch.random.manual_seed(45)
tree = grammar.sample_tree(detach=True)
# Make initial sample feasible
pre_projection_tree = grammar.sample_tree(detach=True)

vis["sample"].delete()
draw_scene_tree_contents_meshcat(pre_projection_tree, zmq_url=vis.window.zmq_url, prefix="sample/pre_projected/contents")
draw_scene_tree_structure_meshcat(pre_projection_tree, zmq_url=vis.window.zmq_url, prefix="sample/pre_projected/structure")

nlp_projected_tree = project_tree_to_feasibility(deepcopy(pre_projection_tree), jitter_q=0.01)
draw_scene_tree_contents_meshcat(nlp_projected_tree, zmq_url=vis.window.zmq_url, prefix="sample/nlp_projected/contents")
draw_scene_tree_structure_meshcat(nlp_projected_tree, zmq_url=vis.window.zmq_url, prefix="sample/nlp_projected/structure")

tree = project_tree_to_feasibility(deepcopy(pre_projection_tree), do_forward_sim=True, zmq_url=vis.window.zmq_url)
draw_scene_tree_contents_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="sample/sim_projected/contents")
draw_scene_tree_structure_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="sample/sim_projected/structure")

Logfile: 

         S N O P T  7.4-1.2  (Feb 2015)
1
 
 SNMEMA EXIT 100 -- finished successfully
 SNMEMA INFO 104 -- memory requirements estimated
1
 
 Parameters

 Files
 -----
 Solution file..........         0       Old basis file ........         0       Standard input.........         5
 Insert file............         0       New basis file ........         0       (Printer)..............        10
 Punch file.............         0       Backup basis file......         0       (Specs file)...........         0
 Load file..............         0       Dump file..............         0       Standard output........         6

 Frequencies
 -----------
 Print frequency........       100       Check frequency........        60       Save new basis map.....       100
 Summary frequency......       100       Factorization frequency        50       Expand frequency.......     10000

 QP subproblems
 --------------
 QPsolver Cholesky......
 Scale tolerance........     0.900       Minor 

Logfile: 

         S N O P T  7.4-1.2  (Feb 2015)
1
 
 SNMEMA EXIT 100 -- finished successfully
 SNMEMA INFO 104 -- memory requirements estimated
1
 
 Parameters

 Files
 -----
 Solution file..........         0       Old basis file ........         0       Standard input.........         5
 Insert file............         0       New basis file ........         0       (Printer)..............        10
 Punch file.............         0       Backup basis file......         0       (Specs file)...........         0
 Load file..............         0       Dump file..............         0       Standard output........         6

 Frequencies
 -----------
 Print frequency........       100       Check frequency........        60       Save new basis map.....       100
 Summary frequency......       100       Factorization frequency        50       Expand frequency.......     10000

 QP subproblems
 --------------
 QPsolver Cholesky......
 Scale tolerance........     0.900       Minor 

In [4]:
# How long does it take to query nonpenetration constraints on this tree?
constraint = NonpenetrationConstraint()
elapsed = []
for k in range(100):
    start = time.time()
    constraint.eval(tree)
    elapsed.append(time.time() - start)
print("Average duration: ", np.mean(elapsed))

Average duration:  0.006467220783233643


In [5]:
samples = do_fixed_structure_hmc_with_constraint_penalties(
    grammar, tree, num_samples=10, subsample_step=1,
    with_nonpenetration=True, zmq_url=vis.window.zmq_url,
    constraints=constraints
)

Warmup:   0%|          | 0/15 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


Sample: 100%|██████████| 15/15 [00:04,  3.66it/s, step size=3.50e-05, acc. prob=0.000]


In [6]:
for k, tree in enumerate(samples[::]):
    draw_scene_tree_contents_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="samples/contents/%d/structure" % k)
    draw_scene_tree_structure_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="samples/structure/%d/structure" % k)